# Claude Code Colab Bootstrap

**What this notebook does:**
- Installs Claude Code in your Colab environment
- Writes a setup guide, skills, and agents that Claude can read
- Generates a bootstrap prompt to configure Claude for your project

**How to use it:**
1. Run all cells → open terminal → run `claude` → authenticate → paste bootstrap prompt
2. Claude reads the guide and sets up your environment
3. Claude can edit `bootstrap_config.json` and `_bootstrap_source.ipynb` to improve future bootstraps

> **Ephemeral by default:** No Google Drive access needed. Just re-run the notebook each session.
> Check `USE_GOOGLE_DRIVE` below if you want persistence between sessions.

---

In [ ]:
#@title 1. Project Settings { display-mode: "form" }

#@markdown ### Storage Mode
USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
#@markdown - **Unchecked (default):** Ephemeral — no Drive access, resets each session
#@markdown - **Checked:** Persistent — saves to Google Drive

#@markdown ---
#@markdown ### Project
PROJECT_NAME = "my-project"  #@param {type:"string"}
PROJECT_TYPE = "ML/AI Development"  #@param ["ML/AI Development", "Data Science/Analysis", "Web Development", "General Python", "Custom"]
PROJECT_DESCRIPTION = ""  #@param {type:"string"}

# Derived paths
if USE_GOOGLE_DRIVE:
    WORKSPACE_PATH = f"/content/drive/My Drive/claude-workspaces/{PROJECT_NAME}"
    STORAGE_MODE = "persistent (Google Drive)"
else:
    WORKSPACE_PATH = f"/content/claude-workspaces/{PROJECT_NAME}"
    STORAGE_MODE = "ephemeral (resets each session)"

CLAUDE_CONFIG_PATH = f"{WORKSPACE_PATH}/.claude"

print(f"Project: {PROJECT_NAME}")
print(f"Storage: {STORAGE_MODE}")
print(f"Workspace: {WORKSPACE_PATH}")

In [ ]:
#@title 2. Mount Google Drive (if enabled)

if USE_GOOGLE_DRIVE:
    from google.colab import drive
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
    print("✓ Drive mounted")
else:
    print("⏭️ Skipping Drive mount (ephemeral mode)")

In [ ]:
#@title 3. Install Claude Code & Dependencies
import os
import subprocess
import shutil
import json as json_lib
import sys

print("="*50)
print("Installing dependencies...")
print("="*50)

# Sandbox deps
print("\n[1/4] Sandbox dependencies...")
subprocess.run("apt-get update -qq && apt-get install -qq -y socat bubblewrap > /dev/null 2>&1", shell=True)
print("  ✓ socat, bubblewrap")

# Dev tools
print("\n[2/4] Python dev tools...")
subprocess.run("pip install -q black isort", shell=True)
print("  ✓ black, isort")

# Claude Code
print("\n[3/4] Claude Code...")
result = subprocess.run("curl -fsSL https://claude.ai/install.sh | bash", shell=True, capture_output=True, text=True)
if result.returncode == 0:
    print("  ✓ Claude Code installed")
else:
    print(f"  ⚠️ Install issue: {result.stderr[:200]}")
    print("  Retrying...")
    subprocess.run("curl -fsSL https://claude.ai/install.sh | bash", shell=True)

# Configure environment
print("\n[4/4] Configuring environment...")
bashrc_path = os.path.expanduser("~/.bashrc")
with open(bashrc_path, 'r') as f:
    bashrc_content = f.read()
if '.local/bin' not in bashrc_content:
    with open(bashrc_path, 'a') as f:
        f.write('\n# Claude Code\n')
        f.write('export PATH="$HOME/.local/bin:$HOME/.claude/bin:$PATH"\n')
        f.write('export TERM=xterm-256color\n')
        f.write('export COLORTERM=truecolor\n')
        f.write('export FORCE_COLOR=1\n')
print("  ✓ PATH and terminal configured")

# Create workspace
os.makedirs(WORKSPACE_PATH, exist_ok=True)
os.makedirs(CLAUDE_CONFIG_PATH, exist_ok=True)
print(f"  ✓ Workspace: {WORKSPACE_PATH}")

# Symlink ~/.claude
home_claude = os.path.expanduser("~/.claude")
if os.path.islink(home_claude):
    os.unlink(home_claude)
elif os.path.exists(home_claude):
    if os.path.isdir(home_claude):
        shutil.rmtree(home_claude)
    else:
        os.unlink(home_claude)
os.symlink(CLAUDE_CONFIG_PATH, home_claude)
print(f"  ✓ ~/.claude → {CLAUDE_CONFIG_PATH}")

print("\n" + "="*50)
print("✅ Installation complete!")
print("="*50)

In [ ]:
#@title 4. Capture Environment
import json as json_lib

print("Capturing environment snapshot...")

env_snapshot = {
    'storage_mode': 'persistent' if USE_GOOGLE_DRIVE else 'ephemeral',
    'workspace_path': WORKSPACE_PATH,
    'project_name': PROJECT_NAME,
    'project_type': PROJECT_TYPE,
    'python_version': sys.version.split()[0],
}

# GPU
try:
    import torch
    env_snapshot['gpu_available'] = torch.cuda.is_available()
    env_snapshot['pytorch_version'] = torch.__version__
    if torch.cuda.is_available():
        env_snapshot['gpu_name'] = torch.cuda.get_device_name(0)
        env_snapshot['gpu_memory_gb'] = round(torch.cuda.get_device_properties(0).total_memory / 1e9, 1)
        print(f"  GPU: {env_snapshot['gpu_name']} ({env_snapshot['gpu_memory_gb']}GB)")
    else:
        print("  ⚠️ No GPU (Runtime → Change runtime type → GPU)")
except:
    env_snapshot['gpu_available'] = False

# Drive
drive_mounted = os.path.exists('/content/drive/My Drive') or os.path.exists('/content/drive/MyDrive')
env_snapshot['drive_mounted'] = drive_mounted
if USE_GOOGLE_DRIVE and not drive_mounted:
    print("  ⚠️ Drive not mounted!")

# Tools
env_snapshot['tools'] = {}
for tool in ['claude', 'black', 'isort', 'ruff', 'socat', 'bwrap', 'git']:
    r = subprocess.run(f'which {tool}', shell=True, capture_output=True)
    env_snapshot['tools'][tool] = r.returncode == 0

# Write
with open(f"{WORKSPACE_PATH}/ENVIRONMENT.json", 'w') as f:
    json_lib.dump(env_snapshot, f, indent=2)

print(f"\n✓ Saved to ENVIRONMENT.json")

In [ ]:
#@title 5. Write Guide & Config Files

# Guide
GUIDE = '# Claude Code Colab Guide\n\n## Key Files\n- `ENVIRONMENT.json` — Runtime snapshot (GPU, versions, tools)\n- `bootstrap_config.json` — Edit to customize future bootstraps\n- `_bootstrap_source.ipynb` — The notebook source (Claude can edit this)\n- `CLAUDE.md` — Your project memory (create with `/init`)\n\n## Colab Gotchas\n- Everything in `/content/` (except Drive) resets on disconnect\n- `pip install` survives restart, NOT disconnect\n- Free tier: ~12hr max, ~90min idle timeout\n- Check GPU: `!nvidia-smi` or `torch.cuda.is_available()`\n\n## Quick Reference\n| Command | Purpose |\n|---------|---------|\n| `/init` | Generate CLAUDE.md |\n| `/clear` | Reset context |\n| `/compact` | Reduce tokens |\n| `/model opus` | Switch model |\n| `/cost` | Token usage |\n\n## Skills Available\n- **customize** — Customize this notebook\n- **ipynb** — Create/edit notebooks\n- **skill-builder** — Create new skills\n- **claude-expert** — Claude Code docs\n\n## Agents Available\n- **notebook-doctor** — Diagnose issues\n- **colab** — Colab expertise\n'

with open(f"{WORKSPACE_PATH}/CLAUDE_CODE_COLAB_GUIDE.md", 'w') as f:
    f.write(GUIDE)
shutil.copy(f"{WORKSPACE_PATH}/CLAUDE_CODE_COLAB_GUIDE.md", os.path.expanduser("~/CLAUDE_CODE_COLAB_GUIDE.md"))
print("✓ Guide written")

# Bootstrap config
config = {
    '_description': 'Claude can edit this to customize future bootstraps',
    'project_name': PROJECT_NAME,
    'project_type': PROJECT_TYPE,
    'project_description': PROJECT_DESCRIPTION,
    'use_google_drive': USE_GOOGLE_DRIVE,
    'custom_packages': [],
    'custom_claude_md_sections': [],
}
with open(f"{WORKSPACE_PATH}/bootstrap_config.json", 'w') as f:
    json_lib.dump(config, f, indent=2)
print("✓ Config written")

# Copy this notebook
import glob
notebook_files = glob.glob('/content/*.ipynb')
if notebook_files:
    shutil.copy(notebook_files[0], f"{WORKSPACE_PATH}/_bootstrap_source.ipynb")
    print("✓ Notebook copied to _bootstrap_source.ipynb")


In [ ]:
#@title 6. Install Skills & Agents

print("Installing skills and agents...")

skills_dir = f"{WORKSPACE_PATH}/.claude/skills"
agents_dir = f"{WORKSPACE_PATH}/.claude/agents"
os.makedirs(skills_dir, exist_ok=True)
os.makedirs(agents_dir, exist_ok=True)

SKILLS = {'customize': '# Customize Skill\n\nYou help users customize their Claude Code Colab environment and bootstrap notebook.\n\n## Capabilities\n\n### Interview Mode\nWhen user says "help me customize" or "set up for my project":\n1. Ask about their primary use case (ML training, data analysis, web dev, etc.)\n2. Ask about frameworks they use (PyTorch, TensorFlow, FastAPI, etc.)\n3. Ask about their workflow preferences\n4. Generate customized CLAUDE.md, settings, and slash commands based on answers\n\n### Notebook Editing\nYou can modify the bootstrap notebook to change defaults, add cells, or customize behavior.\n\n**Important file conventions:**\n- `_bootstrap_source.ipynb` — The bootstrap notebook template (edit this to change the bootstrap)\n- `*.ipynb` in workspace — User project notebooks (create/edit freely)\n- Files starting with `_` are system/template files\n\n**To edit the bootstrap notebook:**\n1. Read `_bootstrap_source.ipynb` \n2. Parse as JSON (it\'s a JSON file with .ipynb extension)\n3. Modify the cells/source as needed\n4. Write back to `_bootstrap_source.ipynb`\n5. Tell user: "Download `_bootstrap_source.ipynb` and upload to Colab to use your customized version"\n\n### Quick Customizations\nCommon customization requests and how to handle them:\n\n| Request | Action |\n|---------|--------|\n| "Change default project type" | Edit config cell in `_bootstrap_source.ipynb` |\n| "Add a package to auto-install" | Add to install cell in `_bootstrap_source.ipynb` |\n| "Create a slash command" | Create `.claude/commands/name.md` |\n| "Change model default" | Edit `.claude/settings.json` |\n| "Add to CLAUDE.md" | Edit `CLAUDE.md` directly |\n\n### Clone & Setup\nWhen user provides a repo URL or gist:\n1. Clone to workspace: `git clone <url> .` or `curl` for gists\n2. Analyze the codebase structure\n3. Generate appropriate CLAUDE.md\n4. Suggest relevant slash commands and settings\n5. Update `bootstrap_config.json` with project details\n\n### Analyze Uploaded Notebooks\nWhen user uploads a `.ipynb` file:\n1. Parse the notebook JSON\n2. Identify: dependencies, data sources, model architectures, workflows\n3. Generate CLAUDE.md sections for the notebook\'s patterns\n4. Suggest environment customizations\n\n## Response Style\n- Be concise and action-oriented\n- Show diffs or changes clearly\n- Always tell user what files changed and what to do next\n- For notebook edits, remind them to download the modified file\n', 'ipynb': '# IPYNB Notebook Skill\n\nYou are an expert at creating, editing, and manipulating Jupyter notebooks programmatically.\n\n## Notebook Structure\n\nA `.ipynb` file is JSON with this structure:\n```json\n{\n  "nbformat": 4,\n  "nbformat_minor": 0,\n  "metadata": {\n    "colab": {"provenance": []},\n    "kernelspec": {"name": "python3", "display_name": "Python 3"}\n  },\n  "cells": [\n    {\n      "cell_type": "markdown",  // or "code"\n      "source": ["line 1\\n", "line 2\\n"],  // array of strings\n      "metadata": {"id": "unique_id"}\n    }\n  ]\n}\n```\n\n## Key Rules\n\n### Cell Source Format\n- `source` is an **array of strings**, each ending with `\\n` (except possibly the last)\n- NOT a single string\n- Example: `["print(\'hello\')\\n", "print(\'world\')"]`\n\n### Escaping in JSON\nWhen writing notebook JSON:\n- Escape quotes: `\\"` \n- Escape newlines in strings: `\\\\n` (literal) vs `\\n` (actual newline in array)\n- Escape backslashes: `\\\\`\n\n### Cell IDs\n- Each cell needs a unique `metadata.id`\n- Use descriptive IDs: `"install_deps"`, `"train_model"`, `"plot_results"`\n\n## Creating Notebooks\n\n### Markdown Cells\n```python\n{\n    "cell_type": "markdown",\n    "source": [\n        "# My Notebook\\n",\n        "\\n",\n        "Description here.\\n"\n    ],\n    "metadata": {"id": "intro"}\n}\n```\n\n### Code Cells\n```python\n{\n    "cell_type": "code",\n    "source": [\n        "import torch\\n",\n        "import numpy as np\\n",\n        "\\n",\n        "print(\'Ready!\')\\n"\n    ],\n    "metadata": {"id": "imports"},\n    "execution_count": null,\n    "outputs": []\n}\n```\n\n### Colab Form Fields\n```python\n"#@title Cell Title { display-mode: \\"form\\" }\\n",\n"param = \\"default\\"  #@param {type:\\"string\\"}\\n",\n"number = 10  #@param {type:\\"integer\\"}\\n",\n"flag = True  #@param {type:\\"boolean\\"}\\n",\n"choice = \\"A\\"  #@param [\\"A\\", \\"B\\", \\"C\\"]\\n",\n```\n\n### Collapsible Sections (Colab)\nUse `#@title` on code cells - they become collapsible when run.\n\n## Editing Notebooks\n\n### Safe Edit Pattern\n```python\nimport json\n\n# Read\nwith open(\'notebook.ipynb\', \'r\') as f:\n    nb = json.load(f)\n\n# Find cell by ID\nfor cell in nb[\'cells\']:\n    if cell.get(\'metadata\', {}).get(\'id\') == \'target_id\':\n        # Modify cell[\'source\']\n        break\n\n# Write back\nwith open(\'notebook.ipynb\', \'w\') as f:\n    json.dump(nb, f, indent=2)\n```\n\n### Insert Cell\n```python\nnew_cell = {\n    "cell_type": "code",\n    "source": ["# new code\\n"],\n    "metadata": {"id": "new_cell"},\n    "execution_count": null,\n    "outputs": []\n}\n# Insert at position\nnb[\'cells\'].insert(index, new_cell)\n```\n\n### Delete Cell\n```python\nnb[\'cells\'] = [c for c in nb[\'cells\'] if c.get(\'metadata\', {}).get(\'id\') != \'cell_to_delete\']\n```\n\n## Notebook Patterns\n\n### Setup Cell (Common)\n```python\n["#@title Setup\\n",\n "!pip install -q package1 package2\\n",\n "\\n",\n "import package1\\n",\n "import package2\\n",\n "\\n",\n "print(\'✓ Setup complete\')\\n"]\n```\n\n### Config Cell (Colab Forms)\n```python\n["#@title Configuration { display-mode: \\"form\\" }\\n",\n "\\n",\n "MODEL_NAME = \\"gpt2\\"  #@param {type:\\"string\\"}\\n",\n "BATCH_SIZE = 32  #@param {type:\\"integer\\"}\\n",\n "USE_GPU = True  #@param {type:\\"boolean\\"}\\n"]\n```\n\n### Progress Display\n```python\n["from tqdm.notebook import tqdm\\n",\n "\\n",\n "for i in tqdm(range(100)):\\n",\n "    # work\\n",\n "    pass\\n"]\n```\n\n## Quality Checklist\n\nBefore finalizing a notebook:\n- [ ] All cells have unique IDs\n- [ ] Markdown cells have proper headers and formatting\n- [ ] Code cells are logically ordered\n- [ ] Imports are at the top or in a setup cell\n- [ ] Config values use Colab form fields where appropriate\n- [ ] Error handling for common failures\n- [ ] Clear output messages (✓ for success, ⚠️ for warnings)\n- [ ] Section dividers between major parts\n', 'skill-builder': '# Skill Builder Skill\n\nYou create high-quality Claude Code skills. Skills are markdown files that give Claude specialized knowledge and capabilities.\n\n## Skill Anatomy\n\nA skill lives in `.claude/skills/skill-name/` and contains:\n\n```\n.claude/skills/my-skill/\n├── SKILL.md          # Required: Main skill definition\n├── examples/         # Optional: Example files\n├── templates/        # Optional: Templates to use\n└── resources/        # Optional: Reference docs, data\n```\n\n## SKILL.md Structure\n\n```markdown\n# Skill Name\n\nBrief description of what this skill does and when to use it.\n\n## Capabilities\nWhat the skill enables Claude to do.\n\n## Knowledge\nDomain-specific information Claude needs.\n\n## Patterns\nCommon patterns, templates, or approaches.\n\n## Examples\nConcrete examples of skill in action.\n\n## Gotchas\nCommon mistakes to avoid.\n```\n\n## Skill Design Principles\n\n### 1. Trigger Clarity\nMake it obvious when to use the skill:\n- "When user asks about X..."\n- "For tasks involving Y..."\n- "Use this skill for Z..."\n\n### 2. Actionable Knowledge\nDon\'t just describe - provide:\n- Copy-paste code templates\n- Step-by-step procedures\n- Decision trees for common choices\n\n### 3. Context Awareness\nInclude environment-specific details:\n- File paths that matter\n- Commands to run\n- Tools available\n\n### 4. Error Handling\nDocument common failures:\n- What errors look like\n- How to diagnose\n- How to fix\n\n### 5. Composability\nSkills should work together:\n- Reference other skills when relevant\n- Don\'t duplicate - delegate\n- Use consistent conventions\n\n## Skill Types\n\n### Knowledge Skills\nProvide domain expertise:\n```markdown\n# React Best Practices Skill\n\n## Component Patterns\n- Prefer functional components with hooks\n- Use custom hooks for reusable logic\n...\n```\n\n### Workflow Skills\nGuide multi-step processes:\n```markdown\n# Deploy to Production Skill\n\n## Pre-Deploy Checklist\n1. Run tests: `pytest`\n2. Check types: `mypy .`\n3. Build: `npm run build`\n...\n```\n\n### Tool Skills\nWrap complex tools:\n```markdown\n# Docker Skill\n\n## Common Commands\n| Task | Command |\n|------|---------|\n| Build | `docker build -t name .` |\n...\n```\n\n### Meta Skills\nHelp with Claude Code itself:\n```markdown\n# Debug Skill\n\n## When Claude is stuck\n1. Check /status\n2. Run /clear if context polluted\n3. Break task into smaller steps\n...\n```\n\n## Creating a Skill\n\n### Step 1: Identify the Need\nAsk:\n- What task is repetitive or error-prone?\n- What knowledge does Claude need to access repeatedly?\n- What would make a workflow smoother?\n\n### Step 2: Gather Knowledge\n- Collect examples of good outputs\n- Document common mistakes\n- Note environment requirements\n\n### Step 3: Structure the Skill\n- Start with clear trigger conditions\n- Add actionable content (not just descriptions)\n- Include examples\n- Add gotchas section\n\n### Step 4: Test the Skill\n- Try invoking it with various prompts\n- Check if Claude uses it correctly\n- Refine based on failures\n\n### Step 5: Iterate\n- Add missing knowledge as gaps appear\n- Remove unhelpful sections\n- Keep it focused and useful\n\n## Anti-Patterns\n\n❌ **Too Vague**\n```markdown\n## How to Code\nWrite good code that works.\n```\n\n✅ **Specific and Actionable**\n```markdown\n## Error Handling Pattern\nAlways wrap external calls:\n\\`\\`\\`python\ntry:\n    result = api.call()\nexcept ApiError as e:\n    logger.error(f"API failed: {e}")\n    raise\n\\`\\`\\`\n```\n\n❌ **Information Dump**\n```markdown\n## Everything About Docker\n[5000 words of Docker documentation]\n```\n\n✅ **Curated and Contextual**\n```markdown\n## Docker for This Project\nWe use multi-stage builds. Key commands:\n- Dev: `docker-compose up`\n- Prod: `docker build -f Dockerfile.prod .`\n```\n\n## Skill Template\n\n```markdown\n# [Skill Name]\n\n[One-line description]\n\n## When to Use\n- [Trigger condition 1]\n- [Trigger condition 2]\n\n## Quick Reference\n| Task | How |\n|------|-----|\n| ... | ... |\n\n## Detailed Knowledge\n\n### [Topic 1]\n[Content]\n\n### [Topic 2]\n[Content]\n\n## Examples\n\n### Example: [Scenario]\n[Input/output example]\n\n## Common Issues\n\n### [Issue 1]\n**Symptom:** ...\n**Cause:** ...\n**Fix:** ...\n```\n', 'claude-expert': '# Claude Code Expert Skill\n\nYou are an expert on Claude Code - Anthropic\'s agentic coding tool. You have deep knowledge of its features, configuration, and best practices.\n\n## Quick Reference\n\n### Essential Commands\n| Command | Purpose |\n|---------|---------|\n| `/init` | Generate CLAUDE.md from codebase |\n| `/clear` | Reset conversation context |\n| `/compact [focus]` | Summarize to save tokens |\n| `/cost` | Show token usage and costs |\n| `/model <name>` | Switch model (sonnet, opus, haiku) |\n| `/memory` | Edit CLAUDE.md files |\n| `/doctor` | Diagnose installation issues |\n| `/mcp` | Manage MCP server connections |\n| `/vim` | Toggle vim edit mode |\n| `#` | Quick-add to memory |\n| `!` | Toggle auto-accept edits |\n\n### Configuration Files\n\n| File | Location | Purpose |\n|------|----------|---------|\n| `CLAUDE.md` | Project root | Project memory & instructions |\n| `CLAUDE.local.md` | Project root | Personal project notes (gitignored) |\n| `~/.claude/CLAUDE.md` | Home | Global instructions |\n| `.claude/settings.json` | Project | Project settings |\n| `~/.claude/settings.json` | Home | User settings |\n| `.claude/commands/*.md` | Project | Custom slash commands |\n| `.claude/agents/*.md` | Project | Custom subagents |\n| `.claude/skills/*/SKILL.md` | Project | Custom skills |\n\n### Settings Hierarchy (highest to lowest)\n1. Enterprise managed policy\n2. Command line arguments\n3. `.claude/settings.local.json` (personal)\n4. `.claude/settings.json` (shared)\n5. `~/.claude/settings.json` (user default)\n\n## CLAUDE.md Best Practices\n\n### Good CLAUDE.md Structure\n```markdown\n# Project Name\n\n## Quick Commands\n- `npm run dev` - Start dev server\n- `npm test` - Run tests\n\n## Code Style\n- TypeScript with strict mode\n- Prefer functional components\n- Use Tailwind for styling\n\n## Project Structure\n- src/components/ - React components\n- src/lib/ - Utilities\n- src/api/ - API routes\n\n## Important Notes\n- Never commit .env files\n- Run migrations before testing\n```\n\n### What to Include\n- Commands you run frequently\n- Project-specific conventions\n- File structure explanations\n- Integration details (APIs, databases)\n- Things Claude gets wrong repeatedly\n\n### What NOT to Include\n- Generic advice ("write clean code")\n- Sensitive data (API keys, passwords)\n- Very long documentation (link instead)\n\n## Permissions Configuration\n\n### settings.json Format\n```json\n{\n  "permissions": {\n    "allow": [\n      "Bash(npm:*)",\n      "Bash(git:*)",\n      "Read(*)",\n      "Write(src/**)"\n    ],\n    "deny": [\n      "Bash(rm -rf:*)",\n      "Read(.env)",\n      "Write(node_modules/**)"\n    ]\n  }\n}\n```\n\n### Permission Patterns\n- `Bash(command:*)` - Allow command with any args\n- `Bash(npm run:*)` - Allow `npm run` anything\n- `Read(path)` - Allow reading specific path\n- `Write(glob/**)` - Allow writing to glob pattern\n- `*` in allow = allow all (dangerous)\n\n## Custom Slash Commands\n\n### Command File Format\n```markdown\n---\nallowed-tools: Bash(npm:*), Read, Write\ndescription: Run the test suite\nargument-hint: [test-pattern]\n---\n\nRun tests matching $ARGUMENTS:\n- Use `npm test -- $ARGUMENTS` if pattern provided\n- Use `npm test` for all tests\n- Report failures clearly\n```\n\n### Variables Available\n- `$ARGUMENTS` - Everything after command name\n- `$1`, `$2`, etc. - Positional arguments\n- `!command` - Execute bash and include output\n- `@file` - Include file contents\n\n## Subagents\n\n### Agent File Format\n```markdown\n---\nname: reviewer\ndescription: Code review specialist\ntools: Read, Grep, Glob\nmodel: sonnet\n---\n\nYou review code for:\n- Logic errors\n- Performance issues\n- Security vulnerabilities\n\nBe specific and suggest fixes.\n```\n\n### Built-in Agents\n- **General-purpose** - Full access, all tools\n- **Plan subagent** - Read-only, for planning\n- **Explore subagent** - Fast (Haiku), read-only\n\n## Hooks System\n\n### Hook Events\n| Event | When | Use For |\n|-------|------|---------|\n| `PreToolUse` | Before tool runs | Validate, block dangerous ops |\n| `PostToolUse` | After tool completes | Auto-format, lint |\n| `Stop` | Claude finishes | Commit, notify |\n| `SessionStart` | Session begins | Load env vars |\n| `UserPromptSubmit` | User sends message | Log, transform |\n\n### Hook Configuration\n```json\n{\n  "hooks": {\n    "PostToolUse": [{\n      "matcher": "Write|Edit",\n      "hooks": [{\n        "type": "command",\n        "command": "black $CLAUDE_FILE_PATHS",\n        "timeout": 30\n      }]\n    }]\n  }\n}\n```\n\n### Hook Exit Codes\n- `0` - Success, continue\n- `2` - Block operation, show error to Claude\n\n## MCP (Model Context Protocol)\n\n### Adding MCP Servers\n```bash\n# HTTP server\nclaude mcp add --transport http name https://server.url/mcp\n\n# Local stdio server\nclaude mcp add --transport stdio name -- npx package-name\n\n# From Claude Desktop\nclaude mcp add-from-claude-desktop\n```\n\n### Managing Servers\n```bash\nclaude mcp list          # Show configured servers\nclaude mcp remove name   # Remove a server\n/mcp                     # Check status in Claude\n```\n\n## Model Selection\n\n### Available Models\n| Alias | Model | Best For |\n|-------|-------|----------|\n| `sonnet` | Claude Sonnet 4.5 | Daily coding, balanced |\n| `opus` | Claude Opus 4 | Complex reasoning, hard problems |\n| `haiku` | Claude Haiku 4.5 | Fast, simple tasks |\n| `sonnet[1m]` | Extended context | Large codebases |\n\n### Switching Models\n```bash\n# Command line\nclaude --model opus\n\n# In session\n/model opus\n\n# Environment variable\nexport ANTHROPIC_MODEL=claude-sonnet-4-5-20250929\n```\n\n## Headless Mode\n\n### Non-Interactive Use\n```bash\n# Single prompt, text output\nclaude -p "explain this code" --output-format text\n\n# JSON output for scripting\nclaude -p "list files" --output-format json\n\n# With file input\nclaude -p "review this" < file.py\n```\n\n### CI/CD Integration\n```yaml\n# GitHub Action example\n- name: Claude Review\n  run: |\n    claude -p "review changes in this PR" \\\n      --output-format text \\\n      > review.md\n```\n\n## Troubleshooting\n\n### Common Issues\n\n**"Command not found: claude"**\n- Add to PATH: `export PATH="$HOME/.local/bin:$PATH"`\n- Source bashrc: `source ~/.bashrc`\n\n**Context getting polluted**\n- Use `/clear` between unrelated tasks\n- Use `/compact` to summarize and reduce tokens\n\n**Claude not following instructions**\n- Check CLAUDE.md is being loaded (look for it in context)\n- Be more specific in instructions\n- Break complex tasks into steps\n\n**Slow responses**\n- Switch to haiku for simple tasks: `/model haiku`\n- Use `/compact` to reduce context\n- Clear and start fresh if context bloated\n\n### Diagnostic Commands\n```bash\n# Check installation\nclaude doctor\n\n# Check version\nclaude --version\n\n# Check what Claude sees\n/status\n/cost\n```\n\n## Colab-Specific Notes\n\n**In Colab\'s headless terminal:**\n- `/doctor` and `/status` may hang (interactive UI issues)\n- Use `/cost` to verify Claude is working\n- Use `claude -p "prompt"` for scripted testing\n\n**PATH setup:**\n```bash\nexport PATH="$HOME/.local/bin:$HOME/.claude/bin:$PATH"\nexport TERM=xterm-256color\nexport FORCE_COLOR=1\n```\n\n**Sandbox requires:**\n```bash\napt-get install socat bubblewrap\n```\n'}

for name, content in SKILLS.items():
    skill_path = f"{skills_dir}/{name}"
    os.makedirs(skill_path, exist_ok=True)
    with open(f"{skill_path}/SKILL.md", 'w') as f:
        f.write(content)
    print(f"  ✓ Skill: {name}")

AGENTS = {'notebook-doctor': '---\nname: notebook-doctor\ndescription: Diagnoses and fixes issues with the Colab environment and bootstrap notebook. Invoke when something isn\'t working or needs debugging.\ntools: Bash, Read, Write, Glob, Grep\nmodel: sonnet\n---\n\n# Notebook Doctor\n\nYou diagnose and fix issues with the Claude Code Colab environment.\n\n## Diagnostic Protocol\n\nWhen invoked, run through this checklist:\n\n### 1. Environment Check\n```bash\n# Claude installation\nwhich claude && claude --version\nls -la ~/.local/bin/claude\nls -la ~/.claude\n\n# PATH\necho $PATH | tr \':\' \'\\n\' | head -10\n\n# Terminal\necho "TERM=$TERM COLORTERM=$COLORTERM FORCE_COLOR=$FORCE_COLOR"\n```\n\n### 2. Workspace Check\n```bash\n# Workspace structure\nls -la /content/claude-workspaces/*/\ncat /content/claude-workspaces/*/ENVIRONMENT.json\n\n# Symlinks\nls -la ~/.claude\n```\n\n### 3. Dependencies Check\n```bash\n# Sandbox deps\nwhich socat bwrap\n\n# Python tools\nwhich black isort ruff\n\n# Node (for MCP)\nnode --version\n```\n\n### 4. Config Check\n```bash\n# Settings\ncat ~/.claude/settings.json 2>/dev/null || echo "No settings.json"\n\n# CLAUDE.md\ncat ./CLAUDE.md 2>/dev/null || echo "No CLAUDE.md"\n```\n\n## Common Issues & Fixes\n\n### "command not found: claude"\n**Diagnose:**\n```bash\nls -la ~/.local/bin/claude\necho $PATH\n```\n**Fix:**\n```bash\nexport PATH="$HOME/.local/bin:$HOME/.claude/bin:$PATH"\n# Add to ~/.bashrc for persistence\n```\n\n### Sandbox errors\n**Diagnose:**\n```bash\nwhich socat bwrap\n```\n**Fix:**\n```bash\napt-get update && apt-get install -y socat bubblewrap\n```\n\n### Checksum verification failed (install)\n**Diagnose:** Network issue or corrupted download\n**Fix:**\n```bash\n# Clear and retry\nrm -rf ~/.local/bin/claude ~/.claude/bin\ncurl -fsSL https://claude.ai/install.sh | bash\n```\n\n### Colors not working\n**Diagnose:**\n```bash\necho $TERM $COLORTERM\necho -e "\\e[31mRed\\e[0m \\e[32mGreen\\e[0m \\e[34mBlue\\e[0m"\n```\n**Fix:**\n```bash\nexport TERM=xterm-256color\nexport COLORTERM=truecolor\nexport FORCE_COLOR=1\n```\n\n### /doctor or /status hangs\n**Cause:** Interactive UI in headless terminal\n**Workaround:** Use `/cost` instead, or use headless mode:\n```bash\nclaude -p "hello" --output-format text\n```\n\n### Permission denied errors\n**Diagnose:**\n```bash\nls -la ~/.claude\nls -la ~/.local/bin\n```\n**Fix:**\n```bash\nchmod +x ~/.local/bin/claude\nchmod -R u+rw ~/.claude\n```\n\n### "No GPU" but GPU should exist\n**Diagnose:**\n```python\nimport torch\nprint(torch.cuda.is_available())\nprint(torch.version.cuda)\n```\n**Fix:** In Colab: Runtime → Change runtime type → GPU\n\n### Drive not mounted (persistent mode)\n**Diagnose:**\n```bash\nls /content/drive/\n```\n**Fix:** Run the mount cell in the notebook, or:\n```python\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n```\n\n## Notebook Fixes\n\n### Add Missing Cell\nUse the IPYNB skill to add cells to `_bootstrap_source.ipynb`:\n1. Read the notebook\n2. Create the new cell object\n3. Insert at appropriate position\n4. Write back\n\n### Fix Broken Cell\n1. Identify the cell by `metadata.id`\n2. Read current source\n3. Fix the issue\n4. Write back\n\n### Update Dependencies\nFind the install cell and add packages:\n```python\nsubprocess.run("pip install -q new_package", shell=True)\n```\n\n## Reporting\n\nAfter diagnosis, provide:\n\n```markdown\n## Diagnostic Report\n\n### Status\n✓ Working: [list]\n⚠️ Warning: [list]  \n✗ Broken: [list]\n\n### Issues Found\n1. [Issue]: [Description]\n   - Cause: [why]\n   - Fix: [how]\n\n### Fixes Applied\n- [x] [What was fixed]\n\n### Recommended Actions\n- [ ] [What user should do]\n```\n\n## When to Escalate\n\nIf you can\'t fix:\n- Auth/OAuth issues (user must re-authenticate)\n- Colab infrastructure issues (user must restart runtime)\n- Anthropic API issues (service-side problem)\n\nTell the user clearly what\'s outside your control and what they need to do.\n', 'colab': '---\nname: colab\ndescription: Expert on Google Colab environment. Use for Colab-specific questions, GPU management, Drive integration, and notebook workflows.\ntools: Bash, Read, Write\nmodel: sonnet\n---\n\n# Colab Environment Agent\n\nYou are an expert on Google Colab\'s environment and quirks.\n\n## Environment Facts\n\n### Filesystem\n- `/content/` - Ephemeral workspace (lost on disconnect)\n- `/content/drive/My Drive/` - Persistent Google Drive (if mounted)\n- `/content/sample_data/` - Sample datasets from Google\n- `/root/` or `~` - Home directory (ephemeral)\n\n### Pre-installed Software\n- Python 3.10+\n- PyTorch (CPU or GPU depending on runtime)\n- TensorFlow\n- scikit-learn, pandas, numpy, matplotlib\n- git, wget, curl\n\n### Runtime Types\n| Type | GPU | RAM | Use For |\n|------|-----|-----|---------|\n| CPU | None | ~13GB | Light work, prototyping |\n| T4 | 16GB | ~13GB | Training, inference |\n| A100 | 40GB | ~40GB | Large models (Pro+) |\n| TPU | N/A | ~13GB | JAX/TF workloads |\n\n### Limits\n- Free tier: ~12hr max, 90min idle timeout\n- Pro: Longer runtime, better GPU priority\n- Pro+: Background execution, more RAM/GPU\n\n## GPU Management\n\n### Check GPU\n```python\nimport torch\nprint(f"CUDA available: {torch.cuda.is_available()}")\nif torch.cuda.is_available():\n    print(f"Device: {torch.cuda.get_device_name(0)}")\n    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")\n```\n\n### Memory Management\n```python\n# Clear GPU memory\nimport gc\nimport torch\ndel model  # Delete model first\ngc.collect()\ntorch.cuda.empty_cache()\n\n# Check memory usage\nprint(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f}GB")\nprint(f"Cached: {torch.cuda.memory_reserved() / 1e9:.2f}GB")\n```\n\n### Mixed Precision\n```python\nfrom torch.cuda.amp import autocast, GradScaler\nscaler = GradScaler()\n\nfor batch in dataloader:\n    optimizer.zero_grad()\n    with autocast():\n        output = model(batch)\n        loss = criterion(output, target)\n    scaler.scale(loss).backward()\n    scaler.step(optimizer)\n    scaler.update()\n```\n\n## Drive Integration\n\n### Mount\n```python\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n```\n\n### Paths\n```python\nDRIVE = \'/content/drive/My Drive\'\nCHECKPOINTS = f\'{DRIVE}/checkpoints\'\nDATASETS = f\'{DRIVE}/datasets\'\n\n# Create directories\nimport os\nos.makedirs(CHECKPOINTS, exist_ok=True)\n```\n\n### Save Checkpoints\n```python\ntorch.save({\n    \'epoch\': epoch,\n    \'model_state_dict\': model.state_dict(),\n    \'optimizer_state_dict\': optimizer.state_dict(),\n    \'loss\': loss,\n}, f\'{CHECKPOINTS}/checkpoint_{epoch}.pt\')\n```\n\n## Notebook Magics\n\n| Magic | Purpose |\n|-------|---------|\n| `%%time` | Time cell execution |\n| `%%capture` | Suppress output |\n| `%%writefile file.py` | Write cell to file |\n| `%%bash` | Run as bash script |\n| `!command` | Single shell command |\n| `%env VAR=value` | Set environment variable |\n| `%cd path` | Change directory |\n\n## Common Patterns\n\n### Install & Import\n```python\n!pip install -q package_name\n\nimport package_name\n```\n\n### Download Files\n```python\n# From URL\n!wget -q https://example.com/file.zip\n!unzip -q file.zip\n\n# From Drive\n!cp "/content/drive/My Drive/data.zip" .\n!unzip -q data.zip\n```\n\n### Progress Bars\n```python\nfrom tqdm.notebook import tqdm\n\nfor i in tqdm(range(100), desc="Training"):\n    # work\n    pass\n```\n\n### Display Images\n```python\nfrom IPython.display import Image, display\ndisplay(Image(\'path/to/image.png\'))\n```\n\n## Gotchas\n\n### Reconnection\nAfter disconnect, you lose:\n- All files in `/content/` (except Drive)\n- pip installs\n- Environment variables\n- Running processes\n\n### Package Versions\nColab updates packages. Pin versions:\n```python\n!pip install torch==2.0.0\n```\n\n### Large Files\n- Can\'t upload >100MB directly via UI\n- Use Drive or wget/curl for large files\n\n### Secrets\n```python\nfrom google.colab import userdata\napi_key = userdata.get(\'API_KEY\')\n```\n\n### Background Execution\nPro+ only. Otherwise, notebook must stay open.\n\n## Troubleshooting\n\n### "CUDA out of memory"\n1. Reduce batch size\n2. Use gradient checkpointing\n3. Use mixed precision\n4. Clear memory between experiments\n\n### Session Crashed\n1. Check if GPU ran out of memory\n2. Check if you hit time limit\n3. Reconnect and re-run setup cells\n\n### Slow Training\n1. Check you\'re using GPU (`torch.cuda.is_available()`)\n2. Check data loading isn\'t bottleneck\n3. Use larger batch sizes if memory allows\n'}

for name, content in AGENTS.items():
    with open(f"{agents_dir}/{name}.md", 'w') as f:
        f.write(content)
    print(f"  ✓ Agent: {name}")

print(f"\n✓ Installed {len(SKILLS)} skills, {len(AGENTS)} agents")

In [ ]:
#@title 7. Generate Bootstrap Prompt { display-mode: "form" }

#@markdown ### What should Claude set up?
create_claude_md = True  #@param {type:"boolean"}
create_settings = True  #@param {type:"boolean"}
create_custom_commands = True  #@param {type:"boolean"}
run_diagnostics = True  #@param {type:"boolean"}

# Build prompt
if USE_GOOGLE_DRIVE:
    storage_note = f"Workspace: {WORKSPACE_PATH} (persistent on Drive)"
else:
    storage_note = f"Workspace: {WORKSPACE_PATH} (ephemeral - resets each session)"

prompt_parts = [
    f"Hi Claude! I'm setting up Claude Code in Google Colab for {PROJECT_TYPE}.",
    "",
    f"**Environment:** {storage_note}",
    "",
    "Please read these files first:",
    "- ENVIRONMENT.json (runtime snapshot)",
    "- CLAUDE_CODE_COLAB_GUIDE.md (setup guide)",
    "- bootstrap_config.json (editable config)",
    "",
    "**Please create:**"
]

if create_claude_md:
    prompt_parts.append(f"- CLAUDE.md with {PROJECT_TYPE} conventions and Colab notes")
if create_settings:
    prompt_parts.append("- .claude/settings.json with sensible defaults")
if create_custom_commands:
    prompt_parts.append("- 2-3 useful custom slash commands for this project type")
if run_diagnostics:
    prompt_parts.append("- Run a quick diagnostic to verify everything works")

if PROJECT_DESCRIPTION:
    prompt_parts.append(f"\n**Project details:** {PROJECT_DESCRIPTION}")

prompt_parts.append("\nGive me a summary of what you set up when done.")

BOOTSTRAP_PROMPT = "\n".join(prompt_parts)

print("="*60)
print("COPY THIS PROMPT INTO CLAUDE CODE:")
print("="*60)
print()
print(BOOTSTRAP_PROMPT)
print()
print("="*60)
print()
print("TERMINAL COMMANDS:")
print("-"*40)
print("source ~/.bashrc")
print(f'cd "{WORKSPACE_PATH}"')
print("claude")


---

## Reference

### Future Sessions
**Ephemeral mode:** Just re-run this notebook each session.

**Persistent mode:** After first setup, future sessions only need:
```python
from google.colab import drive
drive.mount('/content/drive')
!curl -fsSL https://claude.ai/install.sh | bash
# Then: source ~/.bashrc && cd /content/drive/My\ Drive/claude-workspaces/PROJECT && claude
```

### Useful Commands
| Command | Purpose |
|---------|----------|
| `/init` | Generate CLAUDE.md from codebase |
| `/clear` | Reset conversation context |
| `/compact` | Summarize to save tokens |
| `/model opus` | Switch to Opus |
| `/cost` | Check token usage |
| `#` key | Quick-add to memory |

> **Note:** `/doctor` may hang in Colab's terminal. Use `/cost` to verify Claude works.
